<a href="https://colab.research.google.com/github/pauloday/artbot/blob/main/Artbot_Studio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Artbot Studio
by Paul O'Day

>Based on https://colab.research.google.com/drive/1L8oL-vLJXVcRzCFbPwOoMkPKJ8-aYdPN \
Streamlit runner Created by [Manuel Romero](https://twitter.com/mrm8488)

This is the runner for Artbot Studio, which will run in a different window. Here's how to run it:


1.   Go to the 'Runtime' menu, and select 'Change runtime type'. Make sure the accelerator is set to 'GPU'

2.   Run the init cell below by pressing the play button. You'll only have to run this once per session, even after you restart the runtime. It'll take a little while, it has to download at least 2GB of data to Google's servers, but the connection is really fast.

3.   Restart the runtime before running the next cell to get the studio link. Do this through the 'Runtime' menu, or by pressing ctrl-m then the period key. You'll probably have to run this cell twice before getting a URL.

4. Make sure you have a URL before running the next cell - it won't exit so the URL needs to be generated first. If you run it without a URL just stop it and run the URL generator until you get one.

5.   Run the final cell, then go to the URL. There may be some output here, but the settings interface and image previews are in the other tab.

6.   Have fun!


In [ ]:
!pip install -q streamlit
!git clone --branch main https://github.com/pauloday/artbot
!mv artbot/* .
!./init.sh
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

     |████████████████████████████████| 7.9 MB 5.1 MB/s 
     |████████████████████████████████| 75 kB 3.9 MB/s 
     |████████████████████████████████| 111 kB 53.3 MB/s 
     |████████████████████████████████| 170 kB 42.0 MB/s 
     |████████████████████████████████| 4.2 MB 43.7 MB/s 
     |████████████████████████████████| 122 kB 51.6 MB/s 
     |████████████████████████████████| 786 kB 33.4 MB/s 
     |████████████████████████████████| 368 kB 38.4 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit 3.0.19 which is incompatible.
google-colab 1.0.0 requires ipykernel~=4.10, but you have ipykernel 6.0.3 which is incompatible.
google-colab 1.0.0 requires ipython~=5.5.0, but you have ipython 7.25.0 which is incompatibl

**Restart the runtime before running the next cell**. Do this through the 'Runtime' menu, or by pressing ctrl-m then the period key. The cell will probably fail the first time (as in you don't get a link), idk why yet. Just run it until it works, and don't run the following cell until you have a URL. If you do run the last cell before you get a URL, this one will fail the first.time you rerun it.

In [4]:
get_ipython().system_raw('./ngrok http 8501 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    'import sys, json; print("Execute the next cell and the go here: " +json.load(sys.stdin)["tunnels"][0]["public_url"])'

Execute the next cell and the go here: https://3d37411cddd6.ngrok.io


Make sure there's a link to `{something random}.ngrok.io` above before running the next cell

In [ ]:


!streamlit run /content/artbot_studio.py


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://34.141.180.178:8501

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100% 528M/528M [00:03<00:00, 168MB/s]
8.19kB [00:00, 84.5kB/s]       
100%|███████████████████████████████████████| 354M/354M [00:06<00:00, 58.0MiB/s]




Once you're done, stop the above cell (it may hang - it's fine to restart the runtime). Then run this cell to create a zip of all the images you made this session. You can download it (or move it to your Google drive if it's too big) at through the folder icon on the left menu bar. The zip will be called `Gaillery.zip`


In [12]:
import shutil
shutil.make_archive('Gaillery', format='zip', root_dir='Gaillery')

'/content/Gaillery.zip'